In [3]:
%pip install pandas scikit-learn xgboost tensorflow joblib

Note: you may need to restart the kernel to use updated packages.


In [4]:
# train_models.py
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from joblib import dump
import tensorflow as tf

# ------------------------------------------------------------------
# 1. Carga de datos
# ------------------------------------------------------------------
DATA_DIR = Path('.')          # ajusta si lo necesitas
train_df = pd.read_csv(DATA_DIR / 'train.csv')
val_df   = pd.read_csv(DATA_DIR / 'val.csv')
test_df  = pd.read_csv(DATA_DIR / 'test.csv')

target_col  = 'result'

# ------------------------------------------------------------------
# Cargar cadenas de MArkov
# ------------------------------------------------------------------
SPACE = " "
def str_to_vec(s: str) -> np.ndarray:
    """Convierte la cadena '0.123 0.0 …' en vector float32."""
    return np.fromstring(s, sep=SPACE, dtype=np.float32)

def build_feature_matrix(df: pd.DataFrame) -> np.ndarray:
    """
    Output:  [simhash, astsimilarity,  markov1(flat) , markov2(flat) , |diff| ]
              └─2 cols──┘          └────────── 3 × L  ───────────────┘
    """
    base   = df[["simhash", "astsimilarity"]].values.astype("float32")

    m1 = np.vstack(df["markov1"].apply(str_to_vec))
    m2 = np.vstack(df["markov2"].apply(str_to_vec))
    diff = np.abs(m1 - m2)

    return np.hstack([base, m1, m2, diff])

vlen   = len(str_to_vec(train_df["markov1"].iloc[0]))
dim_v  = int(np.sqrt(vlen))
print(f"Vocab Markov detectado: {dim_v} nodos  =>  {vlen} números por matriz")


Vocab Markov detectado: 52 nodos  =>  2704 números por matriz


In [5]:


feature_cols = ["simhash", "astsimilarity"]

X_train = build_feature_matrix(train_df)
X_val   = build_feature_matrix(val_df)
X_test  = build_feature_matrix(test_df)

y_train = train_df[target_col].astype("int8").values
y_val   = val_df  [target_col].astype("int8").values
y_test  = test_df [target_col].astype("int8").values

# ------------------------------------------------------------------
# 2. Pre-procesamiento: escalamos numéricos
# ------------------------------------------------------------------
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_val_sc   = scaler.transform(X_val)
X_test_sc  = scaler.transform(X_test)


In [ ]:

# ------------------------------------------------------------------
# 3. Modelo 0 — Baseline con TensorFlow
# ------------------------------------------------------------------
def build_tf_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(input_dim,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1,  activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
    return model

tf_model = build_tf_model(X_train_sc.shape[1])
tf_model.fit(X_train_sc, y_train,
             validation_data=(X_val_sc, y_val),
             epochs=50, batch_size=256, verbose=0,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

print('\n🟢 TensorFlow — Test metrics')
tf_metrics = tf_model.evaluate(X_test_sc, y_test, verbose=0)
for name, val in zip(tf_model.metrics_names, tf_metrics):
    print(f'{name}: {val:.4f}')
tf_model.save('tf_baseline.keras')


NameError: name 'preprocess' is not defined

In [ ]:

# ------------------------------------------------------------------
# 4. Modelo 1 — Regresión logística
# ------------------------------------------------------------------
logreg = LogisticRegression(max_iter=2000, n_jobs=-1)
logreg.fit(X_train_sc, y_train)
print('\n🟢 LogReg — Val accuracy:',
      accuracy_score(y_val, logreg.predict(X_val_sc)))
dump(('scaler', scaler, 'logreg', logreg), 'logreg.joblib')


In [ ]:

# ------------------------------------------------------------------
# 5. Modelo 2 — Random Forest
# ------------------------------------------------------------------
rf = RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        class_weight='balanced',
        n_jobs=-1,
        random_state=42)
rf.fit(X_train, y_train)                  # RF no necesita escalado
print('\n🟢 RandomForest — Val accuracy:',
      accuracy_score(y_val, rf.predict(X_val)))
dump(rf, 'rf.joblib')


In [ ]:

# ------------------------------------------------------------------
# 6. Modelo 3 — XGBoost
# ------------------------------------------------------------------
xgb_clf = XGBClassifier(
        n_estimators=800,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.9,
        colsample_bytree=0.8,
        objective='binary:logistic',
        eval_metric='logloss',
        n_jobs=-1,
        random_state=42)

xgb_clf.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False)

print('\n🟢 XGBoost — Val accuracy:',
      accuracy_score(y_val, (xgb_clf.predict(X_val) > 0.5)))
xgb_clf.save_model('xgb.json')


In [ ]:

# ------------------------------------------------------------------
# 7. Comparación final en TEST
# ------------------------------------------------------------------
def evaluate(name, y_true, y_pred, y_prob):
    print(f'🔸 {name}: acc={accuracy_score(y_true, y_pred):.4f}',
          f'f1={f1_score(y_true, y_pred):.4f}',
          f'auc={roc_auc_score(y_true, y_prob):.4f}')

print('\n📊  Métricas finales en TEST:')
evaluate('LogReg',
         y_test,
         logreg.predict(X_test_sc),
         logreg.predict_proba(X_test_sc)[:,1])

evaluate('RandomForest',
         y_test,
         rf.predict(X_test),
         rf.predict_proba(X_test)[:,1])

evaluate("XGBoost",
         y_test,
         (xgb_clf.predict(X_test) > 0.5),
         xgb_clf.predict_proba(X_test)[:,1])

tf_pred_prob = tf_model.predict(X_test_sc).ravel()
tf_pred_bin  = (tf_pred_prob > 0.5).astype(int)
evaluate("TensorFlow", y_test, tf_pred_bin, tf_pred_prob)
